# Imports

In [89]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plot

In [2]:
pd.set_option("display.max_column", None)

# Carrega dados

Dados utilizados: https://www.kaggle.com/competitions/walmart-recruiting-store-sales-forecasting/data

## Conjunto base de treino

In [16]:
df_train_base = pd.read_csv("data/train.csv")
df_train_base.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


Para cada Store e Date, é possível ter N valores de Dept, já que as vendas são separadas por Store-Dept-Date. Como vamos realizar a junção deste dataset com os outros através apenas das colunas Store e Date como uma chave única composta, vamos checar se, para um mesmo Dept, a chave Store-Date é única

In [17]:
df_train_base[["Store", "Date", "Dept"]].value_counts()

Store  Date        Dept
1      2010-02-05  1       1
29     2011-03-25  80      1
                   95      1
                   93      1
                   92      1
                          ..
15     2010-10-01  72      1
                   71      1
                   67      1
                   59      1
45     2012-10-26  98      1
Name: count, Length: 421570, dtype: int64

A chave Store-Date é única entre todos os valores de Dept, sendo possível utilizá-la nos joins sem problemas de duplicação de dados

In [126]:
print(f"Intervalo de datas: [{df_train_base['Date'].min()}, {df_train_base['Date'].max()}]")

Intervalo de datas: [2010-02-05, 2012-10-26]


## Conjunto de features

In [6]:
df_features = pd.read_csv("data/features.csv")
df_features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [127]:
print(f"Intervalo de datas: [{df_features['Date'].min()}, {df_features['Date'].max()}]")

Intervalo de datas: [2010-02-05, 2013-07-26]


## Conjunto de stores

In [7]:
df_stores = pd.read_csv("data/stores.csv")
df_stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


## Junção dos conjuntos

In [18]:
df_train_feature = pd.merge(df_train_base, df_features, on=["Store", "Date"], how="left", validate="many_to_one")
df_train_feature.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [111]:
df_full = pd.merge(df_train_feature, df_stores, on=["Store"], how="left", validate="many_to_one") 
df_full.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False,A,151315


## Remoção de campos duplicados

As colunas IsHoliday_x e IsHoliday_y, em teoria, possuem os mesmos valores. Vamos garantir que ambas estão de fato duplicadas e deixar apenas uma

In [112]:
# checa se existem instancias que possuem os valores das colunas IsHoliday_x e IsHoliday_y 
# diferentes entre si
df_full[df_full["IsHoliday_x"] != df_full["IsHoliday_y"]]

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size


In [113]:
# remove uma das colunas duplicadas da informacao IsHoliday e renomeia a outra 
df_full.drop(columns=["IsHoliday_y"], inplace=True)
df_full.rename(columns={"IsHoliday_x": "IsHoliday"}, inplace=True)

# Separação dos dados

Para tentar simular um cenário em que temos os dados de teste rotulados para uma medição da performance final do modelo, vamos separar o conjunto total de dados em treino e teste, utilizando o conjunto de treino para treino e validação via cross-validation e o de teste apenas para a medição final de performance.

In [114]:
print(f"Intervalo de datas: [{df_full['Date'].min()}, {df_full['Date'].max()}]")

Intervalo de datas: [2010-02-05, 2012-10-26]


Vamos utilizar 2 anos para o conjunto de treino e validação (2010-02-05 até 2012-02-04) e o restante para teste (2012-02-05, 2012-10-26)

In [184]:
df_train = df_full[df_full["Date"] <= "2012-02-04"]
df_test = df_full[df_full["Date"] >= "2012-02-05"]

In [185]:
print(f"Intervalo de datas de treino: [{df_train['Date'].min()}, {df_train['Date'].max()}]")
print(f"Intervalo de datas de teste: [{df_test['Date'].min()}, {df_test['Date'].max()}]")

Intervalo de datas de treino: [2010-02-05, 2012-02-03]
Intervalo de datas de teste: [2012-02-10, 2012-10-26]


# Análise Descritiva dos Dados

In [186]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 308959 entries, 0 to 421531
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         308959 non-null  int64  
 1   Dept          308959 non-null  int64  
 2   Date          308959 non-null  object 
 3   Weekly_Sales  308959 non-null  float64
 4   IsHoliday     308959 non-null  bool   
 5   Temperature   308959 non-null  float64
 6   Fuel_Price    308959 non-null  float64
 7   MarkDown1     38527 non-null   float64
 8   MarkDown2     34929 non-null   float64
 9   MarkDown3     36293 non-null   float64
 10  MarkDown4     34312 non-null   float64
 11  MarkDown5     38821 non-null   float64
 12  CPI           308959 non-null  float64
 13  Unemployment  308959 non-null  float64
 14  Type          308959 non-null  object 
 15  Size          308959 non-null  int64  
dtypes: bool(1), float64(10), int64(3), object(2)
memory usage: 38.0+ MB


In [187]:
df_train.describe()

,Store,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size
count,308959.000000,308959.000000,308959.000000,308959.000000,308959.000000,38527.000000,34929.000000,36293.000000,34312.000000,38821.000000,308959.000000,308959.000000,308959.000000
mean,22.180558,44.191132,16028.230955,58.049578,3.218141,6705.718581,7946.238032,5300.414616,3786.708762,5544.450277,169.743148,8.196553,136867.121515
std,12.780538,30.437236,22914.276437,19.044839,0.432690,11805.820775,15441.772632,18152.028839,10038.741362,6174.718284,38.555308,1.865767,60922.444784
min,1.000000,1.000000,-4988.940000,-2.060000,2.472000,0.500000,0.000000,-0.870000,2.000000,135.160000,126.064000,4.261000,34875.000000
25%,11.000000,18.000000,2112.000000,44.130000,2.817000,1003.580000,72.300000,44.160000,169.200000,1798.510000,131.901968,7.241000,93638.000000
50%,22.000000,37.000000,7650.090000,59.120000,3.153000,3312.590000,1402.860000,169.850000,646.200000,3645.000000,182.120157,8.028000,140167.000000
75%,33.000000,72.000000,20209.720000,72.620000,3.594000,6196.770000,7331.950000,659.340000,2084.640000,6418.360000,211.160805,8.625000,202505.000000
max,45.000000,99.000000,693099.360000,100.140000,4.211000,88646.760000,104519.540000,141630.610000,67474.850000,37581.270000,223.900677,14.313000,219622.000000


In [188]:
df_train.dtypes

Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday          bool
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
Type             object
Size              int64
dtype: object

## Limpeza dos dados

### Correção de tipagem das colunas

#### Date

Vamos mudar o formato da coluna Date para datetime

In [189]:
df_train.loc[:, "Date"] = pd.to_datetime(df_train["Date"], format="%Y-%m-%d")

In [190]:
df_train.dtypes

Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday          bool
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
Type             object
Size              int64
dtype: object

### Remoção de dados

#### Weekly_Sales negativo

In [191]:
df_train[df_train["Weekly_Sales"] < 0]

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
6048,1,47,2010-02-19 00:00:00,-863.00,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315
6049,1,47,2010-03-12 00:00:00,-698.00,False,57.79,2.667,NaN,NaN,NaN,NaN,NaN,211.380643,8.106,A,151315
6051,1,47,2010-10-08 00:00:00,-58.00,False,63.93,2.633,NaN,NaN,NaN,NaN,NaN,211.746754,7.838,A,151315
6056,1,47,2011-04-08 00:00:00,-298.00,False,67.84,3.622,NaN,NaN,NaN,NaN,NaN,215.074394,7.682,A,151315
6057,1,47,2011-07-08 00:00:00,-198.00,False,85.83,3.480,NaN,NaN,NaN,NaN,NaN,215.277175,7.962,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419597,45,80,2010-02-12 00:00:00,-0.43,True,27.73,2.773,NaN,NaN,NaN,NaN,NaN,181.982317,8.992,B,118221
419598,45,80,2010-02-19 00:00:00,-0.27,False,31.27,2.745,NaN,NaN,NaN,NaN,NaN,182.034782,8.992,B,118221
419603,45,80,2010-04-16 00:00:00,-1.61,False,54.28,2.899,NaN,NaN,NaN,NaN,NaN,181.692477,8.899,B,118221
419614,45,80,2010-07-02 00:00:00,-0.27,False,76.61,2.815,NaN,NaN,NaN,NaN,NaN,182.318780,8.743,B,118221


Dado que não temos o contexto do negócio para tentar entender melhor estes números negativos de vendas, vamos assumir que os mesmos estão errados e apagá-los da base, já que vendas negativas não parecem fazer sentido e são poucas instâncias.

Existem possibilidades de explicação dos valores negativos, como devoluções de produtos contando negativamente. Mas como não conseguimos confirmar as possíveis causas, seguiremos o caminho de remoção.

In [192]:
df_train = df_train[df_train["Weekly_Sales"] >= 0]

### MarkDown1-5 valores nulos

Segundo a descrição dos dados no Kaggle, as colunas de 1 a 5 de MarkDowns devem ser interpretadas como variáveis anonimizadas, não tendo assim o real significado e comportamento das mesmas. Além disso, é dito nesta mesma descrição que estes dados estão disponíveis apenas a partir de Novembro de 2011, e não estão disponíveis para todas as lojas todo o tempo, guardando o valor NaN nos casos de indisponibilidade.

Como os MarkDowns são dados relacionados à promoções e descontos oferecidos pelas lojas em semanas importantes de vendas durante o ano que precedem grandes feriados e, segundo a descrição dos dados, o acerto no volume de vendas nestas semanas é muito importante e esses dados de MarkDown podem ter um poder preditor relevante, vamos tentar aplicar estratégias de imputação nas colunas em instâncias com dados faltantes.

Vamos iniciar checando se os feriados ocorrem sempre na mesma semana do ano:

In [193]:
# vamos criar uma coluna indicando o numero da semana do ano
df_train["week_number"] = df_train["Date"].apply(lambda x: x.isocalendar()[1])
df_train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,week_number
0,1,1,2010-02-05 00:00:00,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315,5
1,1,1,2010-02-12 00:00:00,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315,6
2,1,1,2010-02-19 00:00:00,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315,7
3,1,1,2010-02-26 00:00:00,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315,8
4,1,1,2010-03-05 00:00:00,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315,9


In [194]:
# vamos verificar se todos os anos, os feriados caem na mesma semana do ano
(
    # primeiro, removemos as linhas duplicadas por week_number e IsHoliday
    df_train[["week_number", "IsHoliday"]].drop_duplicates()
    # apos isso, contamos quantas vezes cada semana aparece. Se uma semana aparecer mais de uma vez
    # significa que em um ano teve feriado na semana e em outro nao
    ["week_number"].value_counts()
)

week_number
5     1
6     1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    1
47    1
48    1
49    1
50    1
51    1
52    1
1     1
2     1
3     1
32    1
31    1
30    1
17    1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
18    1
29    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
4     1
Name: count, dtype: int64

Podemos ver que os feriados caem sempre na mesma semana do ano (week_number). Como os valores de MarkDowns estão diretamente relacionados às semanas e seus feriados, podemos tentar realizar a imputação dos dados de MarkDown nos baseando na week_number.

Como os dados de MarkDowns estão disponíveis a partir de Novembro de 2011 e nossos dados de treino vão até Fevereiro de 2012, não temos mais de um valor de MarkDown por semana na base. Sendo assim, vamos imputar os valores de MarkDown existentes por Store-Dept-week_number nos dados históricos de MarkDown faltantes por Store-Dept-week_number.

In [195]:
# agora criamos um dataframe auxiliar que contem o valor dos markdowns por semana
df_aux_markdown_per_week = df_train.groupby(["Store", "Dept", "week_number"]).agg(
    week_MarkDown1=("MarkDown1", "max"),
    week_MarkDown2=("MarkDown2", "max"),
    week_MarkDown3=("MarkDown3", "max"),
    week_MarkDown4=("MarkDown4", "max"),
    week_MarkDown5=("MarkDown5", "max")
).reset_index()

In [196]:
df_train = pd.merge(df_train, df_aux_markdown_per_week, on=["Store", "Dept", "week_number"], how="left", validate="many_to_one")

In [197]:
df_train["MarkDown1"] = np.where(
    df_train["MarkDown1"].isnull(),
    df_train["week_MarkDown1"],
    df_train["MarkDown1"]
)
df_train["MarkDown2"] = np.where(
    df_train["MarkDown2"].isnull(),
    df_train["week_MarkDown2"],
    df_train["MarkDown2"]
)
df_train["MarkDown3"] = np.where(
    df_train["MarkDown3"].isnull(),
    df_train["week_MarkDown3"],
    df_train["MarkDown3"]
)
df_train["MarkDown4"] = np.where(
    df_train["MarkDown4"].isnull(),
    df_train["week_MarkDown4"],
    df_train["MarkDown4"]
)
df_train["MarkDown5"] = np.where(
    df_train["MarkDown5"].isnull(),
    df_train["week_MarkDown5"],
    df_train["MarkDown5"]
)

In [198]:
# remove colunas dos markdowns por semana, ja que nao sao mais necessarias
df_train.drop(columns=[
    "week_MarkDown1", "week_MarkDown2", "week_MarkDown3", "week_MarkDown4", "week_MarkDown5"
], inplace=True)

In [199]:
df_train.sample(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,week_number
285314,41,93,2011-08-05 00:00:00,36862.47,False,72.19,3.554,NaN,NaN,NaN,NaN,NaN,193.911013,6.901,A,196321,31
112459,16,42,2011-12-16 00:00:00,2912.27,False,22.50,3.266,2801.50,NaN,99.57,422.60,2684.19,195.984169,6.232,B,57197,50
193935,27,82,2010-12-17 00:00:00,53948.01,False,31.62,3.301,7510.92,28.09,1049.30,1160.14,7853.48,136.529281,8.021,A,204184,50
26517,4,54,2011-12-02 00:00:00,99.76,False,38.71,3.176,9762.39,160.94,2260.29,4767.86,28604.20,129.845967,5.143,A,205863,48
254847,37,14,2010-12-24 00:00:00,3913.78,False,59.10,2.886,158.11,NaN,7.50,NaN,1316.88,210.180560,8.476,C,39910,51
31464,5,24,2011-05-20 00:00:00,1112.76,False,71.37,3.907,NaN,NaN,NaN,NaN,NaN,216.302385,6.489,B,34875,20
90459,13,45,2011-03-11 00:00:00,5.97,False,41.28,3.346,NaN,NaN,NaN,NaN,NaN,128.399500,7.470,A,219622,10
29776,5,6,2011-06-03 00:00:00,1190.53,False,83.81,3.699,NaN,NaN,NaN,NaN,NaN,215.838432,6.489,B,34875,22
186685,26,85,2011-05-13 00:00:00,1762.49,False,48.20,4.095,NaN,NaN,NaN,NaN,NaN,134.593000,7.818,A,152513,19
47441,7,46,2010-03-05 00:00:00,11693.36,False,35.86,2.620,NaN,NaN,NaN,NaN,NaN,189.669505,9.014,B,70713,9
